In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx

import cirq
from cirq.contrib.svg import SVGCircuit

In [ ]:
from recirq.quantum_executable import *
from recirq.quantum_runtime import * 
from recirq.executable_result import *
from recirq.executable_result import _execute_flattened

from recirq.otoc.loschmidt import *

In [ ]:
%load_ext snakeviz

In [ ]:
exegroup = get_all_diagonal_rect_executables_flat(max_side_length=2, n_instances=2)
exegroup = [exe for exe in exegroup if exe.problem_topology.n_qubits <10]
exegroup = QuantumExecutableGroup(executables=exegroup[:])

```python
zsamp = cirq.ZerosSampler()
def execute1(exe):
    if not isinstance(exe.measurement, Bitstrings):
        raise NotImplementedError()
        
    if exe.params != tuple():
        raise NotImplementedError()
        
    # something about placing
    
    result = zsamp.run(exe.circuit, repetitions=exe.measurement.n_repetitions)
    return result

class Z2:
    def execute(self, exes):
        for exe in exes:
            execute1(exe)
```

In [ ]:
len(exegroup.executables)

In [ ]:
QCSBackend('rainbow-23').get_sampler_and_device()
QCSBackend('rainbow-23').get_sampler_and_device()

In [ ]:
QCSBackend('rainbow-23').desc

In [ ]:
SimulatorBackend('rainbow-23', noise_strength=1e-3).desc

In [ ]:
#runtime = ZerosSamplerRuntime.from_device_name('syc23')
#runtime = QuantumRuntime(sampler=cirq.DensityMatrixSimulator(noise=cirq.depolarize(1e-3)), device=None)
#runtime = Z2()
runtime = QuantumRuntime(backend=SimulatorBackend('rainbow-23', noise_strength=1e-3))
runtime

In [ ]:
raw_results = runtime.execute(exegroup)
#raw_results

In [ ]:
results = process_results(raw_results)
results

In [ ]:
import pandas as pd

In [ ]:
r = results[0]

In [ ]:
import dataclasses

In [ ]:
dataclasses.fields(r)

In [ ]:
dd = dataclasses.asdict(r)
dd

In [ ]:
# speed improvement: transpose data; only abbrev once
df = pd.DataFrame([dataclasses.asdict(res) for res in results])

# TODO: dataclasses can say how they prefer to be dictified
#df['area'] = df['width'] * df['height']

df

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
def _p2(df):
    gb = df.groupby(['macrocycle_depth']).agg(['mean', 'std']).reset_index()
    plt.errorbar(
        x = gb['macrocycle_depth'],
        y = gb['success_probability', 'mean'],
        yerr = gb['success_probability', 'std'],
        capsize=7,
        label=df.name
    )

def _p(df):
    plt.title(df.name)
    df.groupby(['width', 'height']).apply(_p2)    
    plt.legend(loc='best')
    plt.yscale('log')
    plt.show()

df.groupby(['run_id', 'n_repetitions']).apply(_p)